# Measurement of the signal strength

$$\mathcal{L} (\mu | x) = \prod_{i}^{n} p (x_i | \mu) $$

In [111]:
import json

import pandas as pd
import numpy as np
import torch

from physics.simulation import mcfm
from physics.analysis import zz4l, zz2l2v
from physics.hstar import sigstr
from nsbi import carl

import matplotlib, matplotlib.pyplot as plt

# 0. Define the parameter space

The first thing to do is to define the parameter space that will define the set of hypotheses we wish to test.
For us, this is the signal strength parameter, which we can safely take to be between 0 and 4:

$$ 0 \leq \mu \leq 4 $$

In [113]:
mu_space = torch.linspace(0.0, 4.0, 401)

# 1. Open the "mystery" dataset

A dataset containing events generated according to an unknown (to you) value of $\mu$, has been prepared. We are going to open this dataset and read out for each event its (1) observables, and (2) number of occurrences.

Note: Do you notice anything strange about (2)? In a real LHC dataset, do you think it's likely for there to be two events with *exactly* the same features? What about non-integer-valued occurences of an event? While these are fundamental differences between a simulated and real dataset, they will not matter for our purposes of performing inference on them.

In [ ]:
df = pd.read_csv(...)
features = [...]

kinematics = torch.tensor(df[features])
nevents    = torch.tensor(df['nevents'])

## 2. Evaluating the likelihood: rate term

As already introduced, our likelihood function is composed of two terms, the first of which can be readily evaluated as

$$\mathcal{L}_{\mathrm{rate}}(\mu | \mathcal{D}) = \mathcal{P}(n ; \nu(\mu)) = \frac{\nu^{n}(\mu) e^{-\nu(\mu)}}{n!},$$

where $n$ is the total number of events in the observed dataset, and $\nu(\mu)$ is the expected number of events, which of course depends on the $\mu$-hypothesis.

### 2.(a) Compute the total observed number of events, $n$

Task: Add up all number of events in the dataset to obtaine the total number of events observed.

nu_obs = torch.sum(nevents)

### 2.(b) Compute the total expected number of events, $\nu(\mu = 1)$

The expected number of events is given by the cross section time luminosity:

$$\nu = \sigma \times L$$

where the cross section, of course, depends on the parameter. 
The predicted "visual" cross sections, i.e. after detector acceptance/efficiency effects, of the total $gg\to(h^{\ast}\to)ZZ\to 4\ell$ process, as well as its individual contributions from the signal, background and interference terms, are already calculated and available.
Let's compute the expected number of events under the SM hypothesis, assuming HL-LHC luminosity of $3000\,\mathrm{fb}^{-1}$.

In [ ]:
lumi = 3000.0  # ifb

with open('data/xsec.json', 'r') as f:
    xs = json.load(f)  # fb
    xs_sig_sm = np.prod(xs['sig'])
    xs_bkg_sm = np.prod(xs['bkg'])
    xs_int_sm = np.prod(xs['int'])

nu_sig_sm = xs_sig_sm * lumi
nu_bkg_sm = xs_bkg_sm * lumi
nu_int_sm = xs_int_sm * lumi

$$\nu_{\mathrm{sbi}}(\mu) = \mu \nu_{\mathrm{s}}(1) + \sqrt{\mu} \nu_{\mathrm{i}}(1) + \nu_{\mathrm{b}}(1)$$

In [ ]:
nu_sig_mu = nu_sig_sm * mu_space
nu_int_mu = nu_int_sm * mu_space
nu_sbi_mu = nu_sig_mu + nu_int_mu + nu_bkg_sm

### 2.(d) Define & compute the Poisson likelihood

Using the quantities computed above, we can now compute the negative log likelihood (NLL) of the rate term as a function of $\mu$:

$$- \log \mathcal{L}_{\mathrm{rate}}(\mu | \mathcal{D})  = \nu(\mu) - n \log\nu(\mu) + \log(n!)$$

Reminder: the "disappearance" of $-\log (1/n!)$ term does not affect the minimization of NLL as a function of $\mu$.

In [ ]:
def poisson_negative_log_likelihood(n_obs, nu_exp):
    """""" 

In [116]:
plt.plot(mu_space, t_rate)
plt.show()

## 3. Evaluating the likelihood (ratio): shape term

Here comes NSBI, which will estimate the shape term of our likelihood, and (hopefully) improve our results!

### 3.(a) Load the NN models

Let's first load the CARL models that we've trained in the previous tutorial.

In [118]:
run_dir = 'run/h4l'
_, _, scaler_sbi_over_bkg, model_sbi_over_bkg = carl.utils.load_results(run_dir, 'sbi_over_bkg')
_, _, scaler_sig_over_bkg, model_sig_over_bkg = carl.utils.load_results(run_dir, 'sig_over_bkg')

### 3.(b) Run the models over the dataset, and perform the likelihood ratio trick

This part should also be straightforward, given the previous 

In [1]:
r_sig_over_bkg_sm = carl.utils.get_likelihood_ratio(events, features, scaler_sig_over_bkg, model_sig_over_bkg)
r_sbi_over_bkg_sm = carl.utils.get_likelihood_ratio(events, features, scaler_sbi_over_bkg, model_sbi_over_bkg)

s_sig_over_bkg_sm = model_sig_over_bkg.predict(scaler_sig_over_bkg.transform(events.kinematics[features]))
r_sig_over_bkg_sm = s_sig_over_bkg_sm / (1 - s_sig_over_bkg_sm)

NameError: name 'carl' is not defined

### 3.(c) Evaluate the probability density ratio

$$
\frac{p_{\rm sbi} (x | \mu)}{p_{\rm bkg} (x)} = \frac{ (\mu - \sqrt{\mu}) \sigma_{\rm sig} r_{\rm sig} + \sqrt{\mu} \sigma_{\rm sbi} r_{\rm sbi} + (1-\sqrt{\mu}) \sigma_{\rm bkg} }{ \mu \sigma_{\rm sig} + \sqrt{\mu} \sigma_{\rm int} + \sigma_{\rm bkg} }
$$

Tip: if you want to compute all elements of the $N \times M$ tensor where $N$ is the number of entries in the dataset and $M$ is the number of $\mu$ values being tested, then tensor broadcasting is
```py
a.shape  # (N,)
b.shape  # (M,)
c = a[:,None] * b[None,:]
c.shape  # (N, M)
```

In [ ]:
multiplier_sig = mu_space - torch.sqrt(mu_space)
multiplier_sbi = torch.sqrt(mu_space)
multiplier_bkg = 1 - torch.sqrt(mu_space)

r_sbi_over_bkg_mu = ( xs_sig * multiplier_sig[None,:] * r_sig_over_bkg_sm[:,None] + xs_sbi * multiplier_sbi[None,:] * r_sbi_over_bkg_sm[:,None] + xs_bkg * multiplier_bkg[None,:] ) / (xs_sig * mu_space + xs_int * torch.sqrt(mu_space) + xs_bkg)

Evaluate $t_{shape} = - \sum \log_{i}^{n} (\frac{p_{\rm sbi} (x | \mu)}{p_{\rm bkg} (x)})$

In [ ]:
t_shape = -2 * torch.sum(lhc_lumi * torch.tensor(w_asimov)[:,None] * torch.log(r_sbi_over_bkg_mu), dim=0)

In [ ]:
plt.plot(mu_space, t_shape)
plt.show()
print(torch.min(t_shape))
print(mu_space[torch.argmin(t_shape)])

# Evaluating the likelihood (ratio): rate + shape

$$ t = t_{\rm rate} + t_{\rm shape} $$

In [124]:
t = t_rate + t_shape

In [125]:
plt.plot(mu_space, t)
plt.show()
print(torch.min(t_shape))
print(mu_space[torch.argmin(t_shape)])
print(t[0] - torch.min(t))
print(mu_space[torch.argmin(t)])